In [17]:
with open('../dicts/all_omographs_test.hmg', 'r') as f:
    lines = list(filter(lambda el: not el[0:2] == '||', f.readlines()))

lines = list(map(lambda el: '||' + el, lines))

with open('../dicts/all_omographs_test_out.hmg', 'w') as out:
    out.write(''.join(lines))

In [1]:
import multiprocessing
from joblib import Parallel, delayed
import re
import itertools
import time
import random

num_cores = multiprocessing.cpu_count()

In [28]:
with open("../dicts/all_omographs.hmg", 'r') as f:
    dictEntr = list(f.readlines())
with open('../withErrors/1_firstStep.txt', 'r') as f:
    text = f.read().replace('\xa0', ' ')
text_unsigned = text.replace('!', '.').replace('?', '.').replace('…', '.').replace('–', '-').replace('—', '-').replace('. -', ', -').replace('ё', 'е').replace('Ё', 'Е')


In [4]:
text

'\ufeffв нежелании ехать верхом…\nпрежде его шипы ни в кого не попадали…\nдевушка открыла двери…\nкнопкой стоящего на столе чайника…\nсовмещённый санузел…\nкостёл…\nкаскадёр…\nвоплощённого…\nнас так перевернут…\nобожжёт…\nфиалковые…\nсмягчили выражение лица…\nидём греться и отсыпаться…\nзамолчал, подбирая слова…\nподнялся на верхнюю галерею замка…\nна пол и стены ложились…\nвитая в сладостных мечтах…\nи он осел на пол…\nне было ни тени горечи…\nнепринуждённо ответила нОра…\nчтобы найти самого слабого…\nзасахаренными…\nи получит право голоса…\nот неё самой зависит не все, но многое…\nэто было дурным знаком…\nкровавой десяти летки…\nпридёшься…\nтак что не будь идиотом…\nдо выпадения снега…\nпридётся выискивать тропы…\nэто было знаком окончательного примирения…\nподтвердили…\nдорога в одну стОрону…\nиз обеспеченной семьи…\nу холма на окраине леса…\nвесь Охаонг утопал в снегах…\nжара была не единственной и даже не главной их проблемой…\nв голове была одна лишь жара…\nв ста метрах от гранит

In [14]:
text = "они попадают в организм…\nпродолжить жизнь без тела носителя…"
text_unsigned = text.replace('!', '.').replace('?', '.').replace('…', '.').replace('–', '-').replace('—', '-').replace('. -', ', -').replace('ё', 'е').replace('Ё', 'Е')


In [29]:

def getRegions(start, end):
    before = text_unsigned[max((start - 301), text_unsigned.rfind('\n', 0, start - 1) + 1):(start - 1)]
    after = text_unsigned[end:min((end + 301), text_unsigned.find('\n', end + 1))]
    dotFirst = max(before.rfind('.'), 0)
    dotLast = after.find('.')
    if dotLast == -1:
        dotLast = len(after)

    while len(before) - dotFirst + dotLast < 40:
        dotFirstNext = before.rfind('.', 0, dotFirst)
        dotLastNext = after.find('.', dotLast + 1)
        if dotLastNext == -1 and dotFirstNext == -1:
            break
        if dotLastNext == -1:
            dotFirst = dotFirstNext
        elif dotFirstNext == -1:
            dotLast = dotLastNext
        elif dotLastNext - dotLast > dotFirst - dotFirstNext:
            dotFirst = dotFirstNext
        else:
            dotLast = dotLastNext
            
    return [before[dotFirst:], after[1:dotLast]]
    
    
def findOccur(ind, line):
    pos = []
    if line[0:2] == '||':
        word = line[2:line.find("=")]
        if text.find(word) != -1:
            occur = re.finditer('\\b' + word + '\\b', text)
            pos = list(map(lambda o: [ind, o.start(), getRegions(o.start(), o.end()) ], occur))
    return pos

if __name__ == "__main__":
    allOcs = Parallel(n_jobs = num_cores)(delayed(findOccur)(ind, line) for ind, line in enumerate(dictEntr[:-1]))

allOcs = list(itertools.chain.from_iterable(allOcs))

In [30]:
allOcs

[[775, 1467, ['там живут три', 'семьи']],
 [782, 1813, ['и ты сделал слишком', 'ставку']],
 [1718, 19, ['\ufeffв нежелании ехать', '']],
 [1789, 1502, ['а стены защищали от', '']],
 [1824, 970, ['до', 'добраться до горы']],
 [2064, 355, ['.', 'в сладостных мечтах']],
 [2255, 1920, ['построенных на случай атомной', '']],
 [2562, 544, ['от нее самой зависит не', ' но многое']],
 [3528, 512, ['и получит право', '']],
 [3563, 1041, ['Фирмин мчался к', '']],
 [3563, 1106, ['охотники снова выступили в путь к', '']],
 [3563, 1202, ['в единственной холодной', '']],
 [3572, 1415, ['на другом конце', '']],
 [3594, 990, ['до вечера добраться до', '']],
 [3594, 1131, ['но еще не дойдя до', '']],
 [3594, 1260, ['как у подножия', '']],
 [3831, 82, ['девушка открыла', '']],
 [4272, 746, ['.', 'в одну стОрону']],
 [4454, 1402, ['на', 'конце города']],
 [4493, 1634, ['извлечение пакета из', 'дерева']],
 [4663, 846, ['.', 'была не единственной и даже не главной их проблемой']],
 [4663, 928, ['в голове б

In [7]:
opts = dictEntr[pr[0]][(dictEntr[pr[0]].find('=') + 1):-1].split(',')
scores = [0] * len(opts)
insts = [0] * len(opts)
opt = 1
st = ''

before = dictEntr[pr[0] + 2 + opt*3][(dictEntr[pr[0] + 2 + opt*3].find(':') + 1):-1]
after = dictEntr[pr[0] + 3 + opt*3][(dictEntr[pr[0] + 3 + opt*3].find(':') + 1):-1]
insts[opt] = before.count('|') + after.count('|')
before_log = pr[2][0]
after_log = pr[2][1]

m = re.search(before, " " + pr[2][0], re.I)
print(m)

NameError: name 'pr' is not defined

In [31]:
manager = multiprocessing.Manager()
log_dict = manager.dict({})

def makeDecision(pr, n):
    opts = dictEntr[pr[0]][(dictEntr[pr[0]].find('=') + 1):-1].split(',')
    if n == 0:
        rand = -1
    else:
        rand = random.randint(1, n)
    if dictEntr[pr[0] + 1][0:2] == '||':
        if rand == 1:
            log_dict[len(log_dict)] = pr[2][0] + ' _' + opts[0] + '_ ' + pr[2][1] + '\n'
        pr.append(opts[0])
        return pr
    scores = [0] * len(opts)
    insts = [0] * len(opts)
    opt = -1
    test = 1
    st = ''
    while opt < len(opts):
        line = dictEntr[pr[0] + test]
        if line.find(':') == -1:
            if rand == 1 and opt > -1:
                st += str(scores[opt]) + ': ' + before_log + ' _' + opts[opt] + '_ ' + after_log + '\n'
            opt += 1
            before_log = pr[2][0]
            after_log = pr[2][1]
        else:
            insts[opt] += line.count('|')
            testType = line[:line.find(':')]
            coef = testType.count('+') * 0.5 + 1
            testType = testType.replace('+', '')
            testBody = line[(line.find(':') + 1):-1]
            
            if len(testBody) > 0:  
                if testType == "before":
                    for o in re.finditer(testBody, pr[2][0], re.I):
                        scores[opt] += coef * max((o.end() - o.start())/4, 1) * max((20 - len(pr[2][0]) + o.end()), 1)**2
                        if rand == 1:
                            before_log = before_log[:o.start()] + before_log[o.start():o.end()].upper() + before_log[o.end():]
                elif testType == "after":
                    for o in re.finditer(testBody, pr[2][1], re.I):
                        scores[opt] += coef * max((o.end() - o.start())/4, 1) * max(20 - o.start() + 1, 1)**2
                        if rand == 1:
                            after_log = after_log[:o.start()] + after_log[o.start():o.end()].upper() + after_log[o.end():]
        test += 1

    maxScore = max(scores)
    maxInd = [i for i, j in enumerate(scores) if j == maxScore]
    resInd = maxInd[0]
    for ind in maxInd[1:]:
        if insts[ind] > insts[resInd]:
            resInd = ind
    if rand == 1:
        log_dict[len(log_dict)] = st + opts[resInd] + '\n'            
    pr.append(opts[resInd])
    return pr

if __name__ == "__main__":
    descs = Parallel(n_jobs = num_cores)(delayed(makeDecision)(pr, 1) for pr in allOcs)


In [26]:
descs

[[775, 1467, ['там живут три', 'семьи'], 'большИх'],
 [782, 1813, ['и ты сделал слишком', 'ставку'], 'большУю'],
 [1718, 19, ['\ufeffв нежелании ехать', ''], 'верхОм'],
 [1789, 1502, ['а стены защищали от', ''], 'вЕтра'],
 [1824, 970, ['до', 'добраться до горы'], 'вЕчера'],
 [2064, 355, ['.', 'в сладостных мечтах'], 'витАя'],
 [2255, 1920, ['построенных на случай атомной', ''], 'войнЫ'],
 [2562, 544, ['от нее самой зависит не', ' но многое'], 'всЁ'],
 [3528, 512, ['и получит право', ''], 'гОлоса'],
 [3563, 1041, ['Фирмин мчался к', ''], 'горЕ'],
 [3563, 1106, ['охотники снова выступили в путь к', ''], 'горЕ'],
 [3563, 1202, ['в единственной холодной', ''], 'горЕ'],
 [3572, 1415, ['на другом конце', ''], 'гОрода'],
 [3594, 990, ['до вечера добраться до', ''], 'горЫ'],
 [3594, 1131, ['но еще не дойдя до', ''], 'горЫ'],
 [3594, 1260, ['как у подножия', ''], 'горЫ'],
 [3831, 82, ['девушка открыла', ''], 'двЕри'],
 [4272, 746, ['.', 'в одну стОрону'], 'дорОга'],
 [4454, 1402, ['на', 'конце 

In [32]:
for key in log_dict:
    print(log_dict[key])

0: там живут три _бОльших_ семьи
400.0: там живут ТРИ _большИх_ семьи
большИх

700.0: и ты сделал СЛИШКОМ _большУю_ ставку
0: и ты сделал слишком _бОльшую_ ставку
большУю

0: ﻿в нежелании ехать _вЕрхом_ 
500.0: ﻿в нежелании ЕХАТЬ _верхОм_ 
верхОм

400.0: а стены защищали ОТ _вЕтра_ 
0: а стены защищали от _ветрА_ 
вЕтра

400.0: ДО _вЕчера_ добраться до горы
0: до _вечерА_ добраться до горы
вЕчера

0: . _вИтая_ в сладостных мечтах
96.0: . _витАя_ в сладостных МЕЧТАХ
витАя

0: построенных на случай атомной _вОйны_ 
324.0: построенных НА СЛУЧАЙ атомной _войнЫ_ 
войнЫ

505.75: от нее самой ЗАВИСИТ не _всЁ_  но многое
0: от нее самой зависит не _всЕ_  но многое
всЁ

500.0: и получит ПРАВО _гОлоса_ 
0: и получит право _голосА_ 
гОлоса

0: Фирмин мчался к _гОре_ 
400.0: Фирмин мчался К _горЕ_ 
горЕ

0: охотники снова выступили в путь к _гОре_ 
400.0: охотники снова выступили в путь К _горЕ_ 
горЕ

0: в единственной холодной _гОре_ 
1163.0: в ЕДИНСТВЕННОЙ ХОЛОДНОЙ _горЕ_ 
горЕ

500.0: на друго

In [40]:
allOcs.sort(key = lambda el: el[1])
newText = ''
pos = 0

for pr in allOcs:
    newText += text[pos:pr[1]] + descs[1]
    pos = len(newText)
newText += text[pos:]

In [26]:
pr = allOcs[0]
newText += text[0:4] + descs[0]
newText
pos = len(newText) + 1

In [39]:
text

'они попадают в организм…\nпродолжить жизнь без тела носителя…'

In [41]:
newText

'они тЕладают в организм…\nпродолжить жизнь без тЕла носителя…'

In [60]:
with open('log.txt', 'w') as out:
    for key in log_dict:
        out.write(log_dict[key])
        out.write('\n-----------\n\n')

In [15]:
text = 'все, что они успеют, это выпить на прощание'
pattern = r'все, что'
pattern2 = r'(\bдо\b =?)(\w+ей)'
re.findall(pattern, text)


['все, что']

In [5]:
text = '- Третий закон Голпалотта гласит, - начала Гермиона, - осторожно, там может быть столик. Гласит, что противоядие от составного зелья - не скинь мою книгу, Мерлина ради - не сводится к смеси противоядий - не врежься в камин - от каждого его компонента.'
pattern = r', - начала(( \w+){0,3})\b'
re.findall(pattern, text)
re.sub(pattern, r', - началА \g<1>', text)
#for o in re.finditer(pattern, text):
#    print(o.start())
#    print(o.end())

'- Третий закон Голпалотта гласит, - началА  Гермиона, - осторожно, там может быть столик. Гласит, что противоядие от составного зелья - не скинь мою книгу, Мерлина ради - не сводится к смеси противоядий - не врежься в камин - от каждого его компонента.'

In [2]:
text1 = 'все, проехали'
text2 = 'правильно'
text3 = 'На'

pattern = r'\b(все, проехали)\b'

re.findall(pattern, text1, re.I)

['все, проехали']